# Limpieza del archivo exel con los datos del titanic:

- 1) explorar columnas: visualizar las columnas, tipo de dato, si contienen datos nulos, registros mal formados, valores atipicos o sin sentido, valores faltantes

- 2) datos desordenados,
 
- 3) eliminar columnas con datos no relevantes




In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from progressbar import progressbar as pbar
from local.lib import mlutils
%matplotlib inline

# Analizar que librerias necesito y eliminar el resto de importaciones commit [main 8fb097c]

dataframe = pd.read_excel('Titanic.xlsx')

dataframe.head()


ModuleNotFoundError: No module named 'seaborn'

## Visualizar las columnas y su  tipo de dato

In [ ]:
dataframe.dtypes

## Ahora eliminare las columnas que solo contienen NaN


## Datos nulos 

In [ ]:
to_drop = ['Unnamed: 12',
           'Unnamed: 13',
           'Unnamed: 14',
           'Unnamed: 15',
           'Unnamed: 16',
           'Unnamed: 17',
           'Unnamed: 18',
           'Unnamed: 19',
           'Unnamed: 20',
           'Unnamed: 21',
           'Unnamed: 22',
           'Unnamed: 23',
           'Unnamed: 24',
           'Unnamed: 25']

dataframe.drop(columns=to_drop, inplace=True, axis=1)

In [ ]:
dataframe.head()

## Valores faltantes

In [ ]:
dataframe.isna().sum()

Ahora tenemos en cuenta el motivo por el cual tenemos las columnas que conservamos, para que queremos esos datos, etc...

## Registros mal formados - Valores atipicos
No se observan

La edad es un tipo de dato continuo, el sexo es de tipo categorico, la columna Pclass es de tipo categorico
intonces vamos a visualizar los datos con plots